In [ ]:
#pip install pydub

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import librosa
import IPython.display as ipd
from pydub import AudioSegment

In [ ]:
if sys.version_info[0] < 3:
   # for Python 2
   import cPickle as pickle
else:
   # for Python 3
   import pickle
    
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device=", device)

device= cpu


In [ ]:
def signal2pytorch(x):
  #Function to convert a signal vector x, like a mono audio signal, into a 3-d Tensor that conv1d of Pytorch expects,
  #https://pytorch.org/docs/stable/nn.html
  #Argument x: a 1-d signal as numpy array
  #input x[batch,sample]
  #output: 3-d Tensor X for conv1d input.
  #for conv1d Input: (N,Cin,Lin), Cin: numer of input channels (e.g. for stereo), Lin: length of signal, N: number of Batches (signals) 
  X = np.expand_dims(x, axis=0)  #add channels dimension (here only 1 channel)
  if len(x.shape)==1: #mono:
      X = np.expand_dims(X, axis=0)  #add batch dimension (here only 1 batch)
  X=torch.from_numpy(X)
  X=X.type(torch.Tensor)
  X=X.permute(1,0,2)  #make batch dimension first
  return X

In [ ]:
class Convautoenc(nn.Module):
  def __init__(self):
      super(Convautoenc, self).__init__()
      #Analysis Filterbank with downsampling of N=1024, filter length of 2N, but only N/2 outputs:
      self.conv1=nn.Conv1d(in_channels=1, out_channels=32, kernel_size=2048, stride=1024, padding=1023, bias=True) #Padding for 'same' filters (kernel_size/2-1)

      #Synthesis filter bank:
      self.synconv1=nn.ConvTranspose1d(in_channels=32, out_channels=1, kernel_size=2048, stride=1024, padding=1023, bias=True)

  def encoder(self, x):
      #Analysis:
      x = self.conv1(x)
      y = torch.tanh(x)
      return y
      
  def decoder(self, y):
      #Synthesis:
      xrek= self.synconv1(y)
      return xrek
      
  def forward(self, x):
      y=self.encoder(x)
      #y=torch.round(y/0.125)*0.125
      xrek=self.decoder(y)
      return xrek

In [ ]:
def limpieza(audioin):
  audio, samplerate = audioin.audio, audioin.samplerate
  batch=1
  audio[0,:]/=np.abs(audio[0,:]).max()
  audio[1,:]/=np.abs(audio[1,:]).max()
  X_train=signal2pytorch(audio[0,:]).to(device) #Convert to pytorch format, batch is first dimension    
  X_test=signal2pytorch(audio[1,:]).to(device) #Convert to pytorch format, batch is first dimension

  ##print("Generate Model:")
  model = Convautoenc().to(device)
  ##print('Total number of parameters: %i' % (sum(p.numel() for p in model.parameters() if p.requires_grad)))
  ##print("Def. loss function:")
  loss_fn = nn.MSELoss()  #MSE
  ##loss_fn = nn.L1Loss()
        
  Ypred=model(X_train)
    
  ##Ypred=Ypred.detach()
  outputlen=len(Ypred[0,0,:]) #length of the signal at the output of the network.
  ##print("outputlen=", outputlen)
      
  Y=X_train[:,:,:outputlen]  #the target signal with same length as model output
        
  ##print("Input X.shape=", X_train.shape )
  ##print("Target Y.shape=", Y.shape)
  ##print("Target Y=", Y)
  #print("max(max(Y))=", max(max(max(Y))))
  #print("min(min(Y))=", min(min(min(Y))))
  ##print("Y.type()=", Y.type())
  ##print(Y.device)     

  learning_rate = 1e-4
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)#, betas=(0.9, 0.999))
  """
  try:
      checkpoint = torch.load("audio_autoenc.torch",map_location='cpu')
      model.load_state_dict(checkpoint['model_state_dict'])
      #optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  except IOError:
      print("fresh start")
  """
      
  #optimrandomdir_pytorch.optimizer(model, loss_fn, X, Ypred, iterations=300, startingscale=1.0, endscale=0.0)
  Ypred=model(X_train)
  #Ypred=Ypred.detach()
  ##print("Ypred=", Ypred)
      
  #randdir=True # True for optimization of random direction, False for pytorch optimization
  randdir=False
      
  if randdir==True:
  #optimization of weights using method of random directions:
      optimrandomdir_pytorch.optimizer(model, loss_fn, X_train, Y, iterations=100000, startingscale=0.25, endscale=0.0)
      #--End optimization of random directions------------------------
  else:
      for epoch in range(4000):
          #distortions: shift and noise:
          #"""
          Xlast=X_train[:,:,-1].clone() 
          X_train[:,:,1:]=X_train[:,:,:-1].clone() #round Robbin, shift 1 right
          X_train[:,:,0]=Xlast.clone()
          Ylast=Y[:,:,-1].clone() 
          Y[:,:,1:]=Y[:,:,:-1].clone() #round Robbin, shift 1 right
          Y[:,:,0]=Ylast.clone()
          #"""
          Ypred=model(X_train+torch.randn(X_train.size()).to(device)*0.05)
          #print("Ypred.shape=", Ypred.shape)
          #loss wants batch in the beginning! (Batch, Classes,...)
          #Ypredp=Ypred.permute(1,2,0)
          #Yp=Y.permute(1,0)
          #print("Ypredp.shape=", Ypredp.shape, "Yp.shape=", Yp.shape )
          loss=loss_fn(Ypred, Y)
          ##if epoch%10==0:
              ##print(epoch, loss.item())
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

  """
  torch.save({#'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict()}, "audio_autoenc.torch")
  """
      
  ww = model.cpu().state_dict()   #read obtained weights
  ##print("ww=", ww)
  '''
  #Plot obtained weights:
  plt.figure(figsize=(10,6))
  plt.plot(np.transpose(np.array(ww['conv1.weight'][0:1,0,:])))
  plt.plot(np.transpose(np.array(ww['synconv1.weight'][0:1,0,:])))
  plt.legend(('Encoder Analysis filter 0', 'Decoder Filter 0'))
  plt.xlabel('Sample')
  plt.ylabel('Value')
  plt.title('The Encoder and Decoder Filter Coefficients')
  plt.grid()
  '''

  #Test on training set:
  Xnoise=(X_train+torch.randn(X_train.size()).to(device)*0.05)
  #Xnoise=torch.cat((torch.zeros(1,1,100), Xnoise),dim=-1)
  predictions=model(Xnoise.cpu()) # Make Predictions based on the obtained weights, on training set
  predictions=predictions.detach()
  predictions=np.array(predictions)
  Y=np.array(Y.cpu()) #target
  ##print(Y.shape)
  #print("Y=",Y)
  ##print("predictions.shape=", predictions.shape)
  #convert to numpy:
  #https://discuss.pytorch.org/t/how-to-transform-variable-into-numpy/104/2
  '''
  #Plot target signal and output of autoencoder:
  plt.figure(figsize=(10,8))
  for b in range(batch):
      plt.plot(Y[b,0,:])
      plt.plot(predictions[b,0,:])
      plt.legend(('Target','Predicted'))
      plt.title('The Target and Predicted Signal, batch '+str(b))
      plt.xlabel('Sample')
      plt.show()
  '''
  Xnoise=Xnoise.detach()
  Xnoise=np.array(Xnoise.cpu())
  xnoise=Xnoise[:,0,:]
  #xnoise=np.transpose(xnoise)
  #xnoise=np.clip(xnoise, -1.0,1.0)
      
  xrek=predictions[:,0,:]  #remove unnecessary dimension for playback
  #xrek=np.transpose(xrek)
  #xrek=np.clip(xrek, -1.0,1.0)

  #Test on Verification set:
  X_test_shifted_100 = nn.ConstantPad1d(100, 0)(X_test)
  x=X_test_shifted_100.cpu()+torch.from_numpy((np.random.normal(scale=0.04, size=X_test_shifted_100.shape))).float()
  predictions=model(x) # Make Predictions based on the obtained weights, on verification set
  predictions=predictions.detach()
  predictions=np.array(predictions)

  '''
  plt.figure(figsize=(10,8))
  for b in range(batch):
      plt.plot(np.array(x[b,0,:]))
      plt.plot(predictions[b,0,:])
      plt.legend(('Original','Predicted'))
      plt.title('The Original and Predicted Signal, batch '+str(b))
      plt.xlabel('Sample')
      plt.show()
  '''
  xrek=predictions[:,0,:]

  '''
  #Mostrar audios
  display(ipd.Audio(x[:,0,:], rate=samplerate))
  display(ipd.Audio(xrek, rate=samplerate))
  '''

  audioin.limpio=xrek

In [1]:
#BY: FCruzV10

import requests
import librosa

url_inicio_normal = "https://github.com/JuliethLopez/Clasificador_sarcasmo/blob/main/Audios/Normal/Audio_"
url_inicio_sarcasmo = "https://github.com/JuliethLopez/Clasificador_sarcasmo/blob/main/Audios/Sarcasmo/Audio_"
url_final = ".mp3?raw=true"

class Audio:
    def __init__(self,num=1,sarcasmo=True):
        if sarcasmo:
            url = url_inicio_sarcasmo+str(num_audio)+url_final
        else:
            url = url_inicio_normal+str(num_audio)+url_final
        guardar_mp3(url,num,sarcasmo)
        info = leer_audio(num,sarcasmo)

        self.sarcasmo = sarcasmo
        self.num = num
        self.audio = info[0]
        self.samplerate = info[1]

def guardar_mp3(url,num,sarcasmo):
    audio = requests.get(url)
    if sarcasmo:
        file = open("AudioSarcasmo"+str(num)+".mp3",'wb')
    else:
        file = open("AudioNormal"+str(num)+".mp3",'wb')
    file.write(audio.content)

def leer_audio(num,sarcasmo):
    if sarcasmo:
        audio, samplerate = librosa.load("AudioSarcasmo"+str(num)+".mp3", mono=False, sr=None)
    else:
        audio, samplerate = librosa.load("AudioNormal"+str(num)+".mp3", mono=False, sr=None)
    return audio, samplerate

max_num_sarcasmo = int(input("¿Cuántos audios de sarcasmo quiere?\n"))
max_num_normal = int(input("¿Cuántos audios normales quiere?\n"))
audios_sarcasmo = []
audios_normales = []
for num_audio in range(1,max_num_sarcasmo+1):
    audios_sarcasmo.append(Audio(num=num_audio,sarcasmo=True))
for num_audio in range(1,max_num_normal+1):
    audios_normales.append(Audio(num=num_audio,sarcasmo=False))

¿Cuántos audios de sarcasmo quiere?
1
¿Cuántos audios normales quiere?
2


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
for aud in audios_normales:
  print("Limpiado audio: {}".format(aud.num))
  limpieza(aud)

Limpiado audio: 1
Limpiado audio: 2
Limpiado audio: 3


In [ ]:
display(ipd.Audio(audios_normales[1].audio, rate=audios_normales[1].samplerate))
display(ipd.Audio(audios_normales[1].limpio, rate=audios_normales[1].samplerate))